In [1]:
import os
import pickle
import sys
import warnings
from glob import glob
import re
import datetime
import itertools
from tqdm import tqdm
from google_trans_new import google_translator  
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, DayLocator, DateFormatter
#from mpl_finance import candlestick2_ohlc, volume_overlay
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import lightgbm
#import talib

In [2]:
dataset_dir="../data_dir/"

In [3]:
# 読み込むファイルを定義します。
inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv",
    "stock_price": f"{dataset_dir}/stock_price.csv",
    "stock_fin": f"{dataset_dir}/stock_fin.csv",
    # 本チュートリアルでは使用しないため、コメントアウトしています。
    # "stock_fin_price": f"{dataset_dir}/stock_fin_price.csv.gz",
    "stock_labels": f"{dataset_dir}/stock_labels.csv",
}

# ファイルを読み込みます
dfs = {}
for k, v in inputs.items():
    print(k)
    dfs[k] = pd.read_csv(v)
    # DataFrameのindexを設定します。
    if k == "stock_price":
        dfs[k].loc[:, "datetime"] = pd.to_datetime(
            dfs[k].loc[:, "EndOfDayQuote Date"]
        )
        dfs[k].set_index("datetime", inplace=True)
    elif k in ["stock_fin", "stock_fin_price", "stock_labels"]:
        dfs[k].loc[:, "datetime"] = pd.to_datetime(
            dfs[k].loc[:, "base_date"]
        )
        dfs[k].set_index("datetime", inplace=True)

stock_list
stock_price
stock_fin
stock_labels


In [14]:

def make_growth_rate(df,columns_list):
    codes = df["Local Code"].unique()
    seasons = df["Result_FinancialStatement ReportType"].unique()
    columns = df.columns
    columns = columns.to_list()
    for column in columns_list:
        a = "last "+column
        print(a)
        print(type(columns))
        columns.append(a)
    df_result = pd.DataFrame(index=[], columns=columns)
    # columns_list.append("base_date")
    # columns_list.append("Local Code")
    for code,season in tqdm(itertools.product(codes,seasons),total=len(codes)*len(seasons)):
        #df["last "+column] = 0
        #print(columns_list)
        df_test = df[(df["Local Code"]==code) & (df["Result_FinancialStatement ReportType"]==season)].copy()
        for column in columns_list:
            #print(columns)
            df_test["last "+column] = df_test[column]
            df_test["last "+column] = df_test[column].shift()
            #df = pd.merge(df,df_test[["last "+column,"base_date","Local Code"]],on = ["base_date","Local Code"],how="left")
            #df_ab, df_ac, on='a', how='left'
        #print(df_result)
        #print(df_test)
        df_result = pd.concat([df_result,df_test])
        #print(df_test)
    return df_result
stock_fin = dfs["stock_fin"]
stock_fin_dup = stock_fin[~stock_fin.duplicated(subset=['Local Code', 'Result_FinancialStatement ReportType',"Result_FinancialStatement FiscalYear"],keep='last')]
df1 = make_growth_rate(stock_fin_dup,["Result_FinancialStatement NetSales","Result_FinancialStatement OrdinaryIncome","Result_FinancialStatement TotalAssets","Result_FinancialStatement NetAssets"])
df1[df1["Local Code"]== 9984][["last Result_FinancialStatement NetSales","Result_FinancialStatement NetSales","Result_FinancialStatement ReportType","Result_FinancialStatement FiscalYear"]]

  0%|                                                                              | 13/18555 [00:00<02:24, 128.67it/s]

last Result_FinancialStatement NetSales
<class 'list'>
last Result_FinancialStatement OrdinaryIncome
<class 'list'>
last Result_FinancialStatement TotalAssets
<class 'list'>
last Result_FinancialStatement NetAssets
<class 'list'>


100%|████████████████████████████████████████████████████████████████████████████| 18555/18555 [10:42<00:00, 28.90it/s]


,last Result_FinancialStatement NetSales,Result_FinancialStatement NetSales,Result_FinancialStatement ReportType,Result_FinancialStatement FiscalYear
2016-04-21,NaN,6810206.0,Q3,2016.0
2017-02-08,6810206.0,6581466.0,Q3,2017.0
2018-02-07,6581466.0,6811274.0,Q3,2018.0
2019-02-06,6811274.0,7168452.0,Q3,2019.0
2020-04-30,7168452.0,7089804.0,Q3,2020.0
2016-11-07,NaN,4271834.0,Q2,2017.0
2017-11-06,4271834.0,4411135.0,Q2,2018.0
2018-11-05,4411135.0,4653853.0,Q2,2019.0
2019-11-06,4653853.0,4651724.0,Q2,2020.0
2020-11-09,4651724.0,2630531.0,Q2,2021.0


In [16]:
df1[df1["Local Code"]== 9984][["last Result_FinancialStatement NetSales","Result_FinancialStatement NetSales",
                               "Result_FinancialStatement OrdinaryIncome", "last Result_FinancialStatement OrdinaryIncome",
                               "Result_FinancialStatement TotalAssets", "last Result_FinancialStatement TotalAssets",
                               "Result_FinancialStatement NetAssets", "last Result_FinancialStatement NetAssets",
                               "Result_FinancialStatement ReportType","Result_FinancialStatement FiscalYear"]]

,last Result_FinancialStatement NetSales,Result_FinancialStatement NetSales,Result_FinancialStatement OrdinaryIncome,last Result_FinancialStatement OrdinaryIncome,Result_FinancialStatement TotalAssets,last Result_FinancialStatement TotalAssets,Result_FinancialStatement NetAssets,last Result_FinancialStatement NetAssets,Result_FinancialStatement ReportType,Result_FinancialStatement FiscalYear
2016-04-21,NaN,6810206.0,918214.0,NaN,21497648.0,NaN,3855507.0,NaN,Q3,2016.0
2017-02-08,6810206.0,6581466.0,975257.0,918214.0,24867038.0,21497648.0,3981711.0,3855507.0,Q3,2017.0
2018-02-07,6581466.0,6811274.0,563804.0,975257.0,29412718.0,24867038.0,6568877.0,3981711.0,Q3,2018.0
2019-02-06,6811274.0,7168452.0,1809834.0,563804.0,36459477.0,29412718.0,9286389.0,6568877.0,Q3,2019.0
2020-04-30,7168452.0,7089804.0,1345378.0,1809834.0,39406437.0,36459477.0,9191419.0,9286389.0,Q3,2020.0
2016-11-07,NaN,4271834.0,665625.0,NaN,22896077.0,NaN,3126322.0,NaN,Q2,2017.0
2017-11-06,4271834.0,4411135.0,219021.0,665625.0,27973483.0,22896077.0,5468373.0,3126322.0,Q2,2018.0
2018-11-05,4411135.0,4653853.0,1402089.0,219021.0,33807555.0,27973483.0,7343144.0,5468373.0,Q2,2019.0
2019-11-06,4653853.0,4651724.0,1116897.0,1402089.0,36848701.0,33807555.0,8623626.0,7343144.0,Q2,2020.0
2020-11-09,4651724.0,2630531.0,1441472.0,1116897.0,35648197.0,36848701.0,8420605.0,8623626.0,Q2,2021.0


In [17]:
df1.to_csv("stock_fin_growth_rate.csv",index = None)